<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/examples_agents_with_orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="chevalblanc/gpt-4o-mini"
model_2_id="deepseek-r1:8b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


In [28]:
!pip install openai --quiet

In [22]:
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {model_2_id}")

In [29]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [ ]:
class Response(BaseModel):
   action: str
   input: str
   next_action: str


In [30]:
def call(system_prompt: str, message: str, model: str = modelid ) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
      response_format=Response
  )

  return completion.choices[0].message.parsed

In [34]:
def orchestrator(userinput: str) -> str:

  response_format = {"action":"", "input":"", "next_action":""}

  prompt = """
                Use the context from memory to plan next steps.
                Context:
                []

                You are an expert intent classifier.
                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions:

               Time Tool Agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               calendar agent - Checks your calendar or schedules a date in your calendar at a given time

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the action "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with "respond_to_user" only when there are no agents to select from or there is no next_action
                - You will return the agent name in the form of {response_format}
                - Always return valid JSON like {response_format} and nothing else.
  """
  return call ("You are an expert intent classifier.", prompt.format(userinput=userinput, response_format=response_format))

In [35]:
result = orchestrator ("Please schedule a date in my calendar starting in 1 hour.")
result

'Based on the user\'s input, I suggest using two agents to solve their task:\r\n\r\n1) Time Tool Agent for getting the current time in Asia/Kolkata timezone.\r\n\r\nUsing this information will help us schedule a date with 1 hour gap from the present time which is required by the user. \r\n\r\n2) calendar agent - Checks your calendar or schedules a date in your calendar at a given time\r\n\r\nUsing these two agents we can efficiently execute the task in the following way:\r\n\r\nLet\'s first get the current Asia/Kolkata time using Time Tool Agent.\r\n\r\n{\'action\': \'TimeToolAgent\', \'input\': \'Asia/Kolkata\'}\r\n\r\nOnce we have this information, let try scheduling a date with 1 hour gap from now on user calendar. \r\n\r\n{\'action\': \'calendar agent\', \'input\': f"Please schedule {Time-Tool-Agent-Output} + 1 Hour for an important meeting."}\r\n\r\nAnd that\'s it!\r\n\r\nHere is the final response:\r\n\r\n{\'action\': \'\', \'input\': \'\'}\r\n{\'time\': Time Tool Agent Output, i